In [1]:
import csv
import xml.etree.ElementTree as et
import numpy as np
import pandas as pd
from datetime import datetime as dt

In [2]:
matches = ['f24-4-2013-731767-eventdetails.xml','f24-4-2013-731769-eventdetails.xml','f24-4-2013-731773-eventdetails.xml',
'f24-4-2013-731774-eventdetails.xml','f24-4-2013-731777-eventdetails.xml','f24-4-2013-731779-eventdetails.xml',
'f24-4-2013-731781-eventdetails.xml','f24-4-2013-731783-eventdetails.xml','f24-4-2013-731785-eventdetails.xml',
'f24-4-2013-731786-eventdetails.xml','f24-4-2013-731789-eventdetails.xml','f24-4-2013-731790-eventdetails.xml',
'f24-4-2013-731793-eventdetails.xml','f24-4-2013-731795-eventdetails.xml','f24-4-2013-731798-eventdetails.xml',
'f24-4-2013-731799-eventdetails.xml','f24-4-2013-731801-eventdetails.xml','f24-4-2013-731802-eventdetails.xml',
'f24-4-2013-731806-eventdetails.xml','f24-4-2013-731809-eventdetails.xml','f24-4-2013-731811-eventdetails.xml',
'f24-4-2013-731814-eventdetails.xml','f24-4-2013-731815-eventdetails.xml','f24-4-2013-731817-eventdetails.xml',
'f24-4-2013-731818-eventdetails.xml','f24-4-2013-731820-eventdetails.xml','f24-4-2013-731821-eventdetails.xml',
'f24-4-2013-731822-eventdetails.xml','f24-4-2013-731823-eventdetails.xml','f24-4-2013-731824-eventdetails.xml',
'f24-4-2013-731825-eventdetails.xml','f24-4-2013-731826-eventdetails.xml','f24-4-2013-731827-eventdetails.xml',
'f24-4-2013-731828-eventdetails.xml','f24-4-2013-731829-eventdetails.xml','f24-4-2013-731830-eventdetails.xml']

In [3]:
x_start = []
y_start = []
x_end = []
y_end = []
outcome = []
minute = []
half = []
team = []
for x in matches:
    tree = et.ElementTree(file = x)
    gameFile = tree.getroot()
    team_dict = {gameFile[0].attrib["home_team_id"]: gameFile[0].attrib["home_team_name"],
                 gameFile[0].attrib["away_team_id"]: gameFile[0].attrib["away_team_name"]}

    for game in gameFile:
    
    #Iterate through each event
        for event in game:
        
        #If the event is a pass (ID = 1)
            if event.attrib.get("type_id") == '1':
            
            #To the correct list, append the correct attribute using attrib.get()
                x_start.append(event.attrib.get("x"))
                y_start.append(event.attrib.get("y"))
                outcome.append(event.attrib.get("outcome"))
                minute.append(event.attrib.get("min"))
                half.append(event.attrib.get("period_id"))
                team.append(team_dict[event.attrib.get("team_id")])
            
            #Iterate through each qualifier 
                for qualifier in event:
                
                #If the qualifier is relevant, append the information to the x or y destination lists
                    if qualifier.attrib.get("qualifier_id") == "140":
                        x_end.append(qualifier.attrib.get("value"))
                    if qualifier.attrib.get("qualifier_id") == "141":
                        y_end.append(qualifier.attrib.get("value"))

In [6]:
#Create a list of our 8 columns/lists
column_titles = ["team", "half", "min", "x_start", "y_start", "x_end", "y_end", "outcome"]
            
#Use pd.DataFrame to create our table, assign the data in the order of our columns and give it the column titles above
final_table = pd.DataFrame(data=[team, half, minute, x_start, y_start, x_end, y_end, outcome],
                           index=column_titles)

#Transpose, or flip, the table. Otherwise, our table will run from left to right, rather than top to bottom
final_table = final_table.T

#Show us the top 5 rows of the table
final_table

,team,half,min,x_start,y_start,x_end,y_end,outcome
0,Croatia,1,0,50.1,50.7,53.1,50.0,1
1,Croatia,1,0,53.1,50.0,38.2,52.5,1
2,Croatia,1,0,38.2,53.0,20.9,48.2,1
3,Croatia,1,0,20.9,48.2,9.0,39.1,1
4,Croatia,1,0,8.8,39.2,30.6,0.0,0
...,...,...,...,...,...,...,...,...
36090,Germany,4,123,5.3,51.2,67.7,37.3,0
36091,Argentina,4,123,27.6,65.7,14.6,67.0,1
36092,Argentina,4,123,14.6,67.0,67.9,95.6,0
36093,Germany,4,123,26.1,20.6,42.2,15.0,0


In [7]:
final_table.isnull().sum()

team       0
half       0
min        0
x_start    0
y_start    0
x_end      0
y_end      0
outcome    0
dtype: int64

In [8]:
final_table.describe()

,team,half,min,x_start,y_start,x_end,y_end,outcome
count,36095,36095,36095,36095,36095,36095,36095,36095
unique,28,4,124,979,885,973,884,2
top,Germany,1,45,99.5,0.0,100.0,100.0,1
freq,4755,17604,950,321,920,224,267,28807


In [9]:
final_table['team'].value_counts()

Germany                   4755
Argentina                 3961
Netherlands               3549
Brazil                    3251
Costa Rica                2004
Spain                     1999
Belgium                   1641
Algeria                   1615
England                   1565
Chile                     1507
Italy                     1257
USA                       1061
Australia                  979
Mexico                     788
Uruguay                    787
Greece                     600
Russia                     549
Portugal                   457
France                     451
South Korea                443
Bosnia and Herzegovina     436
Ghana                      423
Switzerland                418
Nigeria                    366
Cameroon                   365
Croatia                    345
Colombia                   331
Iran                       192
Name: team, dtype: int64

In [10]:
final_table.to_csv('passes.csv')